# Comparison of Population Estimates and Methods for Afghanistan 

##  GHS Population dataset
* The GHS population dataset is availabe [here](https://ghsl.jrc.ec.europa.eu/download.php?ds=pop)
* Has 5 year temporal resolution from 1975 to 2030
* The raw global census data harmonized by CIESIN (GPWv4.11) is disaggregated to grid cells
* The population growth rate is also from CIESIN
* GHS-Built-S based on Sentinel/Landsat is used as target for disaggreagtion of population estimates
* The estimates are scaled to match:
    * The total population time series at ’city’ level from the extended database feeding the UN World Urbanisation Prospects 2018
    * The total population time series at country level provided by the UN World Population Prospects 2019

##  WorldPop
* The WorldPop population dataset is available [here](https://hub.worldpop.org/geodata/listing?id=29)
* Random Forest model is used to generate weights for dasymetric redistribution
* The predictirs used include: night time light, topographic variables, land cover and climatic variables
* The output of the model is used as a weighting layer for dasymetric mapping
* Growth rate for rural and urban areas are from UN World Urbanization Prospects Database, 2012 version (UNPD)

## WorldPop Population Adjusted to Match UNPD Estimate
* The WorlPOp UNDP adjusted dataset is available [here](https://hub.worldpop.org/geodata/summary?id=49924)
* The population source is 2010 estimates from UN Urban and Rural Population by Age and Sex (URPAS)
* The Built-Settlement Growth Model  is used to identify unsettled pixels
* Dasymetric redistribution based on Random Forest is used for mapping
* The country total population is adjusted to match the official United Nations population estimates


## Landscan
* The landscan population data is available [here](https://landscan.ornl.gov/)
* Use top-down modeling approach to disaggregate census data
* Spatial data and high resolution imagery are use for multi-variable dasymetric modeling
* The ML models are country specific

## GPW
* The GPW v4 is available [here](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11/data-download)
* Raw census estimates are extrapolated to to get data for target years: 2000, 2005, 2010, 2015 and 2020.
* The estimates are proportionally allocated to ratser cells using a uniform area weighting approach.
* The estimated population have been nationally adjusted to match United Nation’s World Population Prospects: The 2015 Revision (United Nations, 2015).

## NSIA
* The population data is extracted from this [pdf](http://nsia.gov.af:8080/wp-content/uploads/2022/05/%D8%A8%D8%B1%D8%A2%D9%88%D8%B1%D8%AF-%D9%86%D9%81%D9%88%D8%B3-%D8%B3%D8%A7%D9%84-1401_compressed.pdf)
* The 2003 - 05 household listing data was used to estimpate population for 2022 - 23
* The formula used to estimate population is $P_{t} = P_{0}e^{rt}$ where $P_{t}$ is estimated population, $P_{0}$ is the base year population, $r$ is population growth rate and $t$ is time interval

## FlowMinder
* The population data is extratced from this [pdf](https://www.ipcinfo.org/fileadmin/user_upload/ipcinfo/docs/IPC_Afghanistan_AcuteFoodInsec_2022Mar_2022Nov_report.pdf)

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import warnings
warnings.simplefilter(action='ignore')

In [3]:
import os
#os.chdir("/usr/src/app")

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
import luigi

from luigi.util import requires
from kiluigi.tasks import Task, ExternalTask
from kiluigi.targets import CkanTarget, LocalTarget, IntermediateTarget
import param
from luigi.configuration import get_config

In [5]:
#import hvplot.pandas  # noqa

In [6]:
import geoviews.tile_sources as gts
#import hvplot.pandas  # noqa
import geoviews as gv
import panel as pn
import holoviews as hv

In [7]:
import statsmodels.formula.api as smf

In [8]:
class PullFlowMinder(ExternalTask):
    def output(self):
        return CkanTarget(
        dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"},
        resource={"id": "8d8bd8ce-57f1-4746-a91a-a4df43a72cfc"}
        )

In [9]:
@requires(PullFlowMinder)
class FlowMinderPopData(Task):
    def output(self):
        return IntermediateTarget(task=self, timeout=360000)
    
    @staticmethod
    def get_admin_d_pop(x):
        x=x['flow']
        x = x.split(" ")
        name = x[0]
        try:
            pop = int(float(x[1].replace(",", "")))
        except ValueError:
            name = f"{x[0]} {x[1]}"
            pop = int(float(x[2].replace(",", "")))
        return name, pop
    
    def process(self, df):
        df[['admin1', 'flowminder']] = df.apply(lambda x: self.get_admin_d_pop(x), axis=1, result_type='expand')
        df = df.drop("flow", axis=1)
        df['name'] = df['admin1'].str.replace(" Urban", "")
        df = df.groupby(by='name', as_index=False)['flowminder'].sum()
        return df


    def run(self):
        df = pd.read_excel(self.input().get(), header=None, names=["flow"])
        df = self.process(df)
        with self.output().open("w") as out:
            out.write(df)
        

In [10]:
class NSIAPopData(ExternalTask):
    def output(self):
        return IntermediateTarget(task=self, timeout=360000)
    
    def process(self):
        nsia_pop_map = {
            "Badakhshan": 1091.76,
            "Badghis": 569.15,
            "Baghlan": 1053.20,
            "Balkh": 1578.51,
            "Bamyan": 513.19,
            "Daykundi": 534.68,
            "Farah": 583.42,
            "Faryab": 1_150.15,
            "Ghazni": 1_411.38,
            "Ghor": 791.48,
            "Helmand": 1498.48,
            "Hirat": 2234.66,
            "Jawzjan": 625.07,
            "Kabul": 5572.63,
            "Kandahar": 1464.89,
            "Kapisa": 505.50,
            "Khost": 659.10,
            "Kunar": 517.18,
            "Kunduz": 1_184.02,
            "Laghman": 510.93,
            "Logar": 449.81,
            "Nangarhar": 1_769.99,
            "Nimroz": 190.43,
            "Nuristan": 169.58,
            "Paktika": 802.86,
            "Paktya": 633.87,
            "Panjsher": 175.91,
            "Parwan": 764.58,
            "Samangan": 446.10,
            "Sari pul": 643.53,
            "Takhar": 1_133.57,
            "Uruzgan": 451.64,
            "Wardak": 683.54,
            "Zabul": 398.05,
        }
        df = pd.DataFrame([nsia_pop_map]).T
        df = df.reset_index()
        df.columns = ["name", "nsia"]
        df["nsia"] = df["nsia"] * 1000
        return df
    
    def run(self):
        df = self.process()
        with self.output().open("w") as out:
            out.write(df)

In [11]:
class Admin1GeoJson(ExternalTask):
    def output(self):
        return CkanTarget(
        dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"},
        resource={"id": "288a0783-01bd-4e7d-b51d-ddf4431b0874"}
        )

In [12]:
@requires(FlowMinderPopData, NSIAPopData, Admin1GeoJson)
class Admin1GeomToPopData(Task):
    def output(self):
        return IntermediateTarget(task=self, timeout=360000)
    
    def process(self, gdf, df_minder, df_nsia):
        df = df_minder.merge(df_nsia, on='name', how='outer')
        df["name"] = df["name"].replace(
            {"Helmand": "Hilmand", "Sari pul": "Sar-e-Pul", "Wardak": "Maidan Wardak"}
        )
        gdf = gdf.merge(df, on="name", how="outer")
        return gdf
    
    def run(self):
        with self.input()[0].open() as src:
            df_minder = src.read()
        with self.input()[1].open() as src:
            df_nsia = src.read()
        
        gdf = gpd.read_file(self.input()[2].get())
        
        gdf = self.process(gdf, df_minder, df_nsia)
        with self.output().open("w") as out:
            out.write(gdf)
    

In [13]:
class RasterPopData(ExternalTask):
    def output(self):
        return {
            "landscan_2020": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "e0aeee8b-8b28-4394-b6b6-0a9917dc228a"}),
            "landscan_2021": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "b39afc1d-d367-4a67-8c88-2a6b4af5375c"}),
            "world_pop_2020_UNadj": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "ea289d79-b4ac-4da6-8550-38486bc245ff"}),
            "world_pop_2019_UNadj": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "874b00cd-729a-49af-b5b8-e33198ad27a7"}),
            "world_pop_2020": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "a53f74a1-e02f-477c-a0da-86aa904bf9f8"}),
            "world_pop_2019": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "0b19ec59-dc97-4e80-91b7-d7be37f11106"}),
            "gpw_2020": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "99253feb-20bc-4086-9741-95304f050334"}),
            "ghs_2020": CkanTarget(dataset={"id": "37b098b1-0ee9-4fb0-8411-88f3cebd776a"}, resource={"id": "436eadef-947c-4238-a1bc-d21c33db6ff0"}),
        }

In [14]:
@requires(Admin1GeomToPopData, RasterPopData)
class PopZonalStats(Task):
    def output(self):
        return IntermediateTarget(task=self, timeout=360000)
    
    @staticmethod
    def get_zonal_stats(gdf, file_target, var_name):
        try:
            stats = zonal_stats(gdf, file_target.get(), stats="sum", all_touched=False, geojson_out=True)
        except:
            breakpoint()
        gdf = gdf = gpd.GeoDataFrame.from_features(stats)
        gdf = gdf.rename(columns={"sum": var_name})
        return gdf
    
    def run(self):
        with self.input()[0].open() as src:
            gdf = src.read()
        for var_name, file_target in self.input()[1].items():
            gdf = self.get_zonal_stats(gdf, file_target, var_name)
        with self.output().open("w") as out:
            out.write(gdf)

In [ ]:
luigi.build([PopZonalStats()], local_scheduler=True)

[pid 201367] Worker Worker(salt=549178900, workers=1, host=pmburu-ThinkPad-P52, username=pmburu, pid=201367) failed    Admin1GeomToPopData()
Traceback (most recent call last):
  File "/home/pmburu/miniconda3/envs/pop/lib/python3.8/site-packages/luigi/worker.py", line 191, in run
    new_deps = self._run_get_new_deps()
  File "/home/pmburu/miniconda3/envs/pop/lib/python3.8/site-packages/luigi/worker.py", line 133, in _run_get_new_deps
    task_gen = self.task.run()
  File "/tmp/ipykernel_201367/3768918151.py", line 20, in run
    gdf = gpd.read_file(self.input()[2].get())
  File "/home/pmburu/miniconda3/envs/pop/lib/python3.8/site-packages/geopandas/io/file.py", line 203, in _read_file
    return GeoDataFrame.from_features(
  File "/home/pmburu/miniconda3/envs/pop/lib/python3.8/site-packages/geopandas/geodataframe.py", line 587, in from_features
    return GeoDataFrame(rows, columns=columns, crs=crs)
  File "/home/pmburu/miniconda3/envs/pop/lib/python3.8/site-packages/geopandas/geodataf

In [15]:
acc_md = pn.pane.Markdown(
    """
    Population datasets with almost the same total population and correlation close to 1 are:
    
    * world_pop_2020_UNadj, world_pop_2019_UNadj and ghs_2020
    * gpw_2020, world_pop_2020, and world_pop_2019
    * landscan_2021 and landscan_2020
    
    From  similar datasets identified above we select one of them to make easy to visualize that is world_pop_2020_UNadj and gpw_2020
    """, width=800
)

In [22]:
class DatasetOverview(param.Parameterized):
    def plot_total_pop(self):
        df = PopZonalStats().output().open().read()
        var_names = ["flowminder", "nsia", "landscan_2021", "landscan_2020", "world_pop_2020", "world_pop_2019",  "world_pop_2020_UNadj", "world_pop_2019_UNadj","gpw_2020", "ghs_2020"]
        df = pd.melt(df, id_vars=['admin_1'], value_vars=var_names)
        df = df.groupby(by=['variable'], as_index=False)['value'].sum()
        df = df.sort_values(by=['value'], ascending=False)
        df['value'] = df['value'] / 1000
        df = df.set_index("variable")
        plot = df.hvplot.bar(y='value', stacked=False, legend='top_right').opts(width=800, height=500, xrotation=45, ylabel='Population Count (Thousands)', xlabel='', tools=['hover'], title="Total Population")
        return plot
    
    def plot_heatmap(self):
        df = PopZonalStats().output().open().read()
        var_names = ["flowminder", "nsia", "landscan_2021", "landscan_2020", "world_pop_2020", "world_pop_2019",  "world_pop_2020_UNadj", "world_pop_2019_UNadj","gpw_2020", "ghs_2020"]
        corr = df[var_names].corr()
        mask = np.triu(np.ones_like(corr, dtype=bool))
        mask = pd.DataFrame(mask, index=corr.index, columns=corr.columns, dtype=bool)
        plot_data = np.ma.masked_where(np.asarray(mask), corr)
        plot_data[plot_data.mask]=np.nan
        plot_data = pd.DataFrame(plot_data.data, index=corr.index, columns=corr.columns)
        plot = plot_data.hvplot.heatmap(flip_yaxis=True).opts(width=800, height=500, title='Population Correlation Matrix at Admin 1', xrotation=45, yrotation=45)
        return plot
    
    def plot_bars(self):
        df = PopZonalStats().output().open().read()
        var_names = ["flowminder", "nsia", "landscan_2021",  "world_pop_2020_UNadj","gpw_2020"]
        df = pd.melt(df, id_vars=['admin_1'], value_vars=var_names)
        df = df.sort_values(by=['value'], ascending=False)
        df['value'] = df['value'] / 1000
        plot = df.hvplot.bar(x='admin_1', y='value', by='variable', stacked=False, ylim=(0, 7_000), legend='top_right').opts(width=1600, height=500, xrotation=90, multi_level=False, ylabel='Population Count (Thousands)', xlabel='Admin 1', shared_axes=False, title="Population At Admin 1 Level")
        return plot
        
viewer = DatasetOverview(name="Population Dataset Overview")
pn.Column(pn.Row(viewer.plot_total_pop, viewer.plot_heatmap), pn.Row(acc_md), pn.Row(viewer.plot_bars))

Column
    [0] Row
        [0] ParamMethod(method)
        [1] ParamMethod(method)
    [1] Row
        [0] Markdown(str, width=800)
    [2] Row
        [0] ParamMethod(method)

In [25]:
class ExplanatoryAnalysis(param.Parameterized):
    
    baseline_pop_data = param.ObjectSelector(default="landscan_2021", objects=["flowminder", "nsia", "landscan_2021",  "world_pop_2020_UNadj","gpw_2020"])
    
    pop_data = param.ObjectSelector(default="flowminder", objects=["flowminder", "nsia", "landscan_2021",  "world_pop_2020_UNadj","gpw_2020"])
    
    @pn.depends("baseline_pop_data", "pop_data")
    def plot_baseline(self):
        df = PopZonalStats().output().open().read()
        levels = [40_000, 200_000, 400_00, 500_000, 600_000, 700_000, 800_000, 1_000_000, 1_400_000, 1_600_000, 4_000_000, 7_000_000]
        all_vars_set = {"flowminder", "nsia", "landscan_2021",  "world_pop_2020_UNadj","gpw_2020"}
        vdims = [self.baseline_pop_data, "admin_1"] + list(set(all_vars_set) - {self.baseline_pop_data})
        plot = gv.Polygons(df, vdims=vdims).opts(tools=["hover"], colorbar=True, height=400, width=700, clabel="People", title=self.baseline_pop_data, cmap='Reds', color_levels=levels)
        plot = gts.OSM * plot
        return plot
    
    @pn.depends("baseline_pop_data", "pop_data")
    def plot_scenario(self):
        df = PopZonalStats().output().open().read()
        levels = [40_000, 200_000, 400_00, 500_000, 600_000, 700_000, 800_000, 1_000_000, 1_400_000, 1_600_000, 4_000_000, 7_000_000]
        vdims = [self.pop_data, "admin_1"]
        plot = gv.Polygons(df, vdims=vdims).opts(tools=["hover"], colorbar=True, height=400, width=700, clabel="People", title=self.pop_data, cmap='Reds', color_levels=levels)
        plot = gts.OSM * plot
        return plot
    
    @pn.depends("baseline_pop_data", "pop_data")
    def plot_diff(self):
        df = PopZonalStats().output().open().read()
        df['diff'] = df[self.baseline_pop_data] - df[self.pop_data]
        vdims = ['diff', 'admin_1']
        plot = gv.Polygons(df, vdims=vdims).opts(tools=["hover"], colorbar=True, height=400, width=700, clabel="People", title=f"{self.baseline_pop_data} - {self.pop_data}", cmap='seismic', clim=(-800_000, 800_000))
        plot = gts.OSM * plot
        return plot
    
    @pn.depends("baseline_pop_data", "pop_data")
    def plot_per_diff(self):
        df = PopZonalStats().output().open().read()
        df['pct_change'] = ((df[self.pop_data] - df[self.baseline_pop_data]) / df[self.baseline_pop_data]) * 100
        vdims = ['pct_change', 'admin_1']
        plot = gv.Polygons(df, vdims=vdims).opts(tools=["hover"], colorbar=True, height=400, width=700, clabel="%", title=f"Pct Change From {self.baseline_pop_data} to {self.pop_data}", cmap='seismic', clim=(-100, 100))
        plot = gts.OSM * plot
        return plot
    @pn.depends("baseline_pop_data", "pop_data")
    def plot_bar(self):
        df = PopZonalStats().output().open().read()
        df = pd.melt(df, id_vars=['admin_1'], value_vars=[self.baseline_pop_data, self.pop_data])
        df = df.sort_values(by=['value'], ascending=False)
        df['value'] = df['value']/1000
        plot = df.hvplot.bar(x='admin_1', y='value', by='variable', stacked=False, legend='top_right').opts(width=700, height=400, xrotation=90, multi_level=False, ylabel='Population Count (Thousands)', xlabel='Admin 1')
        return plot
    
viewer = ExplanatoryAnalysis(name="Explanatory Analysis Explorer")
pn.Row(viewer.param, pn.Column(pn.Row(viewer.plot_baseline, viewer.plot_scenario), pn.Row(viewer.plot_bar, viewer.plot_per_diff)))
#pn.Column(pn.Row(viewer.param, viewer.plot_baseline, viewer.plot_scenario), pn.Row(viewer.plot_per_diff, viewer.plot_bar))

Row
    [0] Column(margin=5, name='Explanatory A..., width=300)
        [0] StaticText(value='<b>Explanatory A...)
        [1] Select(name='Baseline pop data', options=OrderedDict([('flowminder'...]), value='landscan_2021')
        [2] Select(name='Pop data', options=OrderedDict([('flowminder'...]), value='flowminder')
    [1] Column
        [0] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
        [1] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)

In [27]:
class PopulationScatterPlot(param.Parameterized):
    
    baseline_pop_data = param.ObjectSelector(default="flowminder", objects=["flowminder", "nsia", "landscan_2021",  "world_pop_2020_UNadj","gpw_2020"])
    
    def scatter_plot(self, df, var='nsia'):
        lm = smf.ols(formula=f'{self.baseline_pop_data} ~ {var}', data=df).fit()
        text = hv.Text(2_000, 5_500, f'{round(lm.params[0], 2)} + {round(lm.params[1], 4)}*{var} \n R squared = {round(lm.rsquared, 4)}')
        
        df[f'{var}_pred'] = lm.predict(df[var])
        #df[f'{var}_pred'] = df[f'{var}_pred']/1000
        ylabel=f"{self.baseline_pop_data} Population Count (Thousands)"
        xlabel=f"{var} Population Count (Thousands)"
        plot = hv.Scatter(df, var, self.baseline_pop_data).opts(size=10, tools=["hover"], height=400, width=600, ylabel=ylabel, xlabel=xlabel) * text * df.hvplot(kind='line', x=var, y=f'{var}_pred', color='black', width=1, title=f"{self.baseline_pop_data} vs {var} Population at Admin 1")
        return plot
    
    @pn.depends("baseline_pop_data")
    def viz(self):
        df = PopZonalStats().output().open().read()
        df = df.drop('geometry', axis=1)
        df = pd.DataFrame(df)
        df = df.sort_values(by=self.baseline_pop_data)
        
        vars_list = list(
            set(["flowminder", "nsia", "landscan_2021",  "world_pop_2020_UNadj","gpw_2020"]) - {self.baseline_pop_data}
        )
        for i in vars_list + [self.baseline_pop_data]:
             df[i] = df[i]/1000
                
        plot_1 = self.scatter_plot(df, vars_list[0])
        plot_2 = self.scatter_plot(df, vars_list[1])
        plot_3 = self.scatter_plot(df, vars_list[2])
        plot_4 = self.scatter_plot(df, vars_list[3])
        viz = pn.Column(pn.Row(plot_1, plot_2), pn.Row(plot_3, plot_4))
        return viz
        

viewer = PopulationScatterPlot(name="Population Scatter Plot")
pn.Row(viewer.param, viewer.viz)

Row
    [0] Column(margin=5, name='Population Scatter Plot', width=300)
        [0] StaticText(value='<b>Population S...)
        [1] Select(name='Baseline pop data', options=OrderedDict([('flowminder'...]), value='flowminder')
    [1] ParamMethod(method)

## Next steps
* Identify key insights
* Add population data from AsiaPop and GRUMP

## References